In [147]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.polars
import os


Kaggle API copy command

(for kaggle notebook)
kaggle datasets download -d mexwell/e-w-brown-solar-facility

Kaggle site

[kaggle data set](https://www.kaggle.com/datasets/mexwell/e-w-brown-solar-facility)

In [148]:
df_1_pd = pd.read_csv("BS_2016.csv")
df_2_pd = pd.read_csv("BS_2017.csv")
df_3_pd = pd.read_csv("BS_2018.csv")
df_4_pd = pd.read_csv("BS_2019.csv")
df_5_pd = pd.read_csv("BS_2020.csv")
df_6_pd = pd.read_csv("BS_2021.csv")
df_7_pd = pd.read_csv("BS_2022.csv")

# Combine all the dataframes using pandas
df_pd = pd.concat([df_1_pd, df_2_pd, df_3_pd, df_4_pd, df_5_pd, df_6_pd, df_7_pd])
df_pd.head()

,Timestamp,Year,Month,Day,Hour,Minute,kW,kWh,POAI,GHI,TmpF,TmpC
0,2016-09-01 00:00:00,2016,9,1,0,0,-35.425629,7746365.5,-0.462653,-0.644379,71.541014,21.967230
1,2016-09-01 00:01:00,2016,9,1,0,1,-35.362911,7746365.5,-0.461384,-0.645051,71.528453,21.960252
2,2016-09-01 00:02:00,2016,9,1,0,2,-35.300190,7746365.5,-0.460114,-0.645723,71.515895,21.953275
3,2016-09-01 00:03:00,2016,9,1,0,3,-35.237469,7746365.5,-0.458845,-0.646395,71.503334,21.946297
4,2016-09-01 00:04:00,2016,9,1,0,4,-35.174748,7746365.5,-0.457575,-0.647067,71.490774,21.939319


In [149]:
# Evaluate the shape of the combined dataframe - it is "small" big data
df_pd.shape

(3330720, 12)

In [150]:
# import the combined dataframe into polars with the from_pandas method
polars_df = pl.from_pandas(df_pd)

In [151]:
# polars has a .head() just like pandas
polars_df.head()

Timestamp,Year,Month,Day,Hour,Minute,kW,kWh,POAI,GHI,TmpF,TmpC
str,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
"""2016-09-01 00:00:00""",2016,9,1,0,0,-35.425629,7746365.5,-0.462653,-0.644379,71.541014,21.96723
"""2016-09-01 00:01:00""",2016,9,1,0,1,-35.362911,7746365.5,-0.461384,-0.645051,71.528453,21.960252
"""2016-09-01 00:02:00""",2016,9,1,0,2,-35.30019,7746365.5,-0.460114,-0.645723,71.515895,21.953275
"""2016-09-01 00:03:00""",2016,9,1,0,3,-35.237469,7746365.5,-0.458845,-0.646395,71.503334,21.946297
"""2016-09-01 00:04:00""",2016,9,1,0,4,-35.174748,7746365.5,-0.457575,-0.647067,71.490774,21.939319


In [152]:
#demonstrate lazy frame
lazy_df = pl.LazyFrame(polars_df)

In [153]:
# lazy frame has a .head() just like polars and pandas, however it shows the query plan. The SLICE is the head representation in the query plan
lazy_df.head()

In [154]:
# printing an entire lazy frame will show the query plan
print(lazy_df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

DF ["Timestamp", "Year", "Month", "Day"]; PROJECT */12 COLUMNS; SELECTION: None


In [155]:
# using collect will execute the query plan and return the result as a polars DataFrame
lazy_df.collect().head()

Timestamp,Year,Month,Day,Hour,Minute,kW,kWh,POAI,GHI,TmpF,TmpC
str,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
"""2016-09-01 00:00:00""",2016,9,1,0,0,-35.425629,7746365.5,-0.462653,-0.644379,71.541014,21.96723
"""2016-09-01 00:01:00""",2016,9,1,0,1,-35.362911,7746365.5,-0.461384,-0.645051,71.528453,21.960252
"""2016-09-01 00:02:00""",2016,9,1,0,2,-35.30019,7746365.5,-0.460114,-0.645723,71.515895,21.953275
"""2016-09-01 00:03:00""",2016,9,1,0,3,-35.237469,7746365.5,-0.458845,-0.646395,71.503334,21.946297
"""2016-09-01 00:04:00""",2016,9,1,0,4,-35.174748,7746365.5,-0.457575,-0.647067,71.490774,21.939319


The below cell is an intended error

In [156]:
# Lazyframes do not have a shape attribute
lazy_df.shape

AttributeError: 'LazyFrame' object has no attribute 'shape'

In [ ]:
# polars naturally splits the data into chunks for parallel processing. This makes it easy to work with big data
# and conduct unified operations.
polars_df.n_chunks(strategy="all")

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [157]:
# lazy frames can use the .columns attribute to show the columns
lazy_df.columns

['Timestamp',
 'Year',
 'Month',
 'Day',
 'Hour',
 'Minute',
 'kW',
 'kWh',
 'POAI',
 'GHI',
 'TmpF',
 'TmpC']

In [158]:
filter_cols = [
    'Timestamp',
    'Year',
    'Month',
    'Day',
    'Hour',
    'Minute',
    'kW',
    'kWh',
    'TmpF'
]

In [159]:
# filter the lazy frame for the year 2016 and month of September
df_2016 = lazy_df.select(filter_cols)
df_2016 = df_2016.filter(pl.col("Year") == 2016)
df_2016 = df_2016.filter(pl.col("Month") == 9)

# alternatively the .select and .filter methods can be chained. However with especially large data sets its best to break it up

In [160]:
# Observe how the query plan has changed to account for the new operations
df_2016.head()

In [161]:
print(df_2016)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

FILTER [(col("Month")) == (9)] FROM
  FILTER [(col("Year")) == (2016)] FROM
     SELECT [col("Timestamp"), col("Year"), col("Month"), col("Day"), col("Hour"), col("Minute"), col("kW"), col("kWh"), col("TmpF")] FROM
      DF ["Timestamp", "Year", "Month", "Day"]; PROJECT */12 COLUMNS; SELECTION: None


In [162]:
# A good practice is to add an snake case dataframe type to the end of the variable name to keep track of the dataframe we're working with
# There are many times where you may want to bounce in and out of lazy frames and polars dataframes and pandas dataframes.
df_2016_pl = df_2016.collect()
df_2016_Lp = df_2016

In [163]:
df_2016_pl.head()

Timestamp,Year,Month,Day,Hour,Minute,kW,kWh,TmpF
str,i64,i64,i64,i64,i64,f64,f64,f64
"""2016-09-01 00:00:00""",2016,9,1,0,0,-35.425629,7746365.5,71.541014
"""2016-09-01 00:01:00""",2016,9,1,0,1,-35.362911,7746365.5,71.528453
"""2016-09-01 00:02:00""",2016,9,1,0,2,-35.30019,7746365.5,71.515895
"""2016-09-01 00:03:00""",2016,9,1,0,3,-35.237469,7746365.5,71.503334
"""2016-09-01 00:04:00""",2016,9,1,0,4,-35.174748,7746365.5,71.490774


In [181]:
# create a histogram of the kW column
df_2016_pl.hvplot.hist("TmpF")

# notice that the ploting is really a pointer to hvplot. We would get the same result with .plot.hist("TmpF")


:Histogram   [TmpF]   (TmpF_count)

In [165]:
df_2016_pl.plot.scatter("kW", 'TmpF')

:Scatter   [kW]   (TmpF)

In [166]:
df_2016_pl.describe()

statistic,Timestamp,Year,Month,Day,Hour,Minute,kW,kWh,TmpF
str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""43200""",43200.0,43200.0,43200.0,43200.0,43200.0,43200.0,43200.0,43200.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,2016.0,9.0,15.5,11.5,29.5,2639.204337,NaN,NaN
"""std""",null,0.0,0.0,8.655542,6.922267,17.318303,3686.202924,NaN,NaN
"""min""","""2016-09-01 00:00:00""",2016.0,9.0,1.0,0.0,0.0,-70.122162,7746365.5,49.462485
"""25%""",null,2016.0,9.0,8.0,6.0,15.0,-33.172257,8307463.5,66.25761
"""50%""",null,2016.0,9.0,16.0,12.0,30.0,19.978291,8.835398e6,72.060655
"""75%""",null,2016.0,9.0,23.0,17.0,44.0,5407.603027,9.29732e6,80.693574
"""max""","""2016-09-30 23:59:00""",2016.0,9.0,30.0,23.0,59.0,10209.8125,9.658889e6,92.324774


In [167]:
df_2016_Lp.describe()

statistic,Timestamp,Year,Month,Day,Hour,Minute,kW,kWh,TmpF
str,str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""43200""",43200.0,43200.0,43200.0,43200.0,43200.0,43200.0,43200.0,43200.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,2016.0,9.0,15.5,11.5,29.5,2639.204337,NaN,NaN
"""std""",null,0.0,0.0,8.655542,6.922267,17.318303,3686.202924,NaN,NaN
"""min""","""2016-09-01 00:00:00""",2016.0,9.0,1.0,0.0,0.0,-70.122162,7746365.5,49.462485
"""25%""",null,2016.0,9.0,8.0,6.0,15.0,-33.172257,8307463.5,66.25761
"""50%""",null,2016.0,9.0,16.0,12.0,30.0,19.978291,8.835398e6,72.060655
"""75%""",null,2016.0,9.0,23.0,17.0,44.0,5407.603027,9.29732e6,80.693574
"""max""","""2016-09-30 23:59:00""",2016.0,9.0,30.0,23.0,59.0,10209.8125,9.658889e6,92.324774


In [168]:
# drop missing values from temperature column
df_2016_pl = df_2016_pl.drop_nulls(subset=['TmpF'])

In [169]:
# continue observing the changes to the query plan
df_2016_Lp = df_2016_Lp.drop_nulls(subset=['TmpF'])

In [170]:
print(df_2016_Lp)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

FILTER col("TmpF").is_not_null().cast(Boolean) FROM
  FILTER [(col("Month")) == (9)] FROM
    FILTER [(col("Year")) == (2016)] FROM
       SELECT [col("Timestamp"), col("Year"), col("Month"), col("Day"), col("Hour"), col("Minute"), col("kW"), col("kWh"), col("TmpF")] FROM
        DF ["Timestamp", "Year", "Month", "Day"]; PROJECT */12 COLUMNS; SELECTION: None


In [171]:
# create a boolean using with_columns to determine if the temperature is above 70
df_2016_pl = df_2016_pl.with_columns((pl.col('TmpF') > 70).alias('TmpF > 70'))
df_2016_Lp = df_2016_Lp.with_columns((pl.col('TmpF') > 70).alias('TmpF > 70'))

In [172]:
df_2016_pl.head()

Timestamp,Year,Month,Day,Hour,Minute,kW,kWh,TmpF,TmpF > 70
str,i64,i64,i64,i64,i64,f64,f64,f64,bool
"""2016-09-01 00:00:00""",2016,9,1,0,0,-35.425629,7746365.5,71.541014,true
"""2016-09-01 00:01:00""",2016,9,1,0,1,-35.362911,7746365.5,71.528453,true
"""2016-09-01 00:02:00""",2016,9,1,0,2,-35.30019,7746365.5,71.515895,true
"""2016-09-01 00:03:00""",2016,9,1,0,3,-35.237469,7746365.5,71.503334,true
"""2016-09-01 00:04:00""",2016,9,1,0,4,-35.174748,7746365.5,71.490774,true


In [173]:
print(df_2016_Lp)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

 WITH_COLUMNS:
 [[(col("TmpF")) > (70.0)].alias("TmpF > 70")], [] 
  FILTER col("TmpF").is_not_null().cast(Boolean) FROM
    FILTER [(col("Month")) == (9)] FROM
      FILTER [(col("Year")) == (2016)] FROM
         SELECT [col("Timestamp"), col("Year"), col("Month"), col("Day"), col("Hour"), col("Minute"), col("kW"), col("kWh"), col("TmpF")] FROM
          DF ["Timestamp", "Year", "Month", "Day"]; PROJECT */12 COLUMNS; SELECTION: None


In [174]:
# value count of the boolean column
df_2016_pl['TmpF > 70'].value_counts()

TmpF > 70,count
bool,u32
false,16678
true,26522


In [182]:
# Dictionaries are a great way to store lazy query plans in a low memory footprint for later use.
# A good practice can be to procedurally split up the data into smaller dataframes and store them in a dictionary so that we can get to a state where we can .collect() them
df_dict = {}
df_dict['2016'] = df_2016_Lp

In [183]:
print(df_dict)

{'2016': <LazyFrame at 0x2366B2C3920>}


In [184]:
df_dict['2016'].collect().head()

Timestamp,Year,Month,Day,Hour,Minute,kW,kWh,TmpF,TmpF > 70
str,i64,i64,i64,i64,i64,f64,f64,f64,bool
"""2016-09-01 00:00:00""",2016,9,1,0,0,-35.425629,7746365.5,71.541014,true
"""2016-09-01 00:01:00""",2016,9,1,0,1,-35.362911,7746365.5,71.528453,true
"""2016-09-01 00:02:00""",2016,9,1,0,2,-35.30019,7746365.5,71.515895,true
"""2016-09-01 00:03:00""",2016,9,1,0,3,-35.237469,7746365.5,71.503334,true
"""2016-09-01 00:04:00""",2016,9,1,0,4,-35.174748,7746365.5,71.490774,true


In [176]:
# writing to parquet is a great way to store data in a columnar format that is easy to read and write and use less space
polars_df.write_parquet("solar_data.parquet")

In [177]:
# Get the size of the parquet file in MB
os.path.getsize("solar_data.parquet") / (1024 * 1024)

97.82985782623291

In [178]:
file_size_sum = 0
# Sum the size of all csvs
for year in range(2016, 2023):
    file_size_sum += os.path.getsize(f"BS_{year}.csv") / (1024 * 1024)

# Sum in MB
file_size_sum

327.54063510894775